# Import Packages

In [33]:
import pandas as pd

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset

import torchinfo
from torch_snippets import Report
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.has_mps:
    device = "mps"

# Read Dataset

In [31]:
general = pd.read_csv("../datasets/general.csv")
room = pd.read_csv("../datasets/room.csv")
categorical = pd.read_csv("../datasets/categorical.csv")
labels = pd.read_csv("../datasets/label.csv")

In [32]:
general

,Postcode,Sale or Let,Price Qualifier,DESC Council Tax Band,# of Enquiry or viewings,# of Apps/Offers
0,LU7 4WN,Sale,Offers In Excess Of,Band B,32,12
1,DA17 5PJ,Sale,Guide Price,Band D,14,4
2,RG26 5PX,Sale,Guide Price,Band E,10,2
3,BD8 0HT,Sale,Offers in Region Of,Band B,9,1
4,HU17 7AB,Sale,Offers Over,Band B,1,2
...,...,...,...,...,...,...
1019,SS9 3BX,Rental,Monthly,Band D,0,0
1020,SS7 3EL,Sale,Offers Over,Band D,0,1
1021,SS7 5PY,Sale,Offers Over,Band F,0,0
1022,CM1 2BD,Rental,Monthly,Band A,0,2


In [27]:
room

,bedroom number,kitchen number,living number,bathroom number,dining number,other number,other area
0,2,1,1,1,1,3,0.0
1,3,1,0,2,1,3,0.0
2,3,1,1,0,1,2,17.2
3,3,1,1,1,1,4,19.5
4,3,1,0,0,1,8,36.9
...,...,...,...,...,...,...,...
1019,3,1,2,2,0,1,8.7
1020,3,1,1,1,1,3,186.2
1021,4,1,0,2,1,6,166.1
1022,1,1,0,1,1,1,3.6


In [28]:
categorical

,Allocated,Communal,Covered,Driveway,Garage,Gated,Off Street,On Street,Permit,Private,...,Gas Central,Night Storage,Oil,Solar,Solar Water,Under Floor,Good,Major renovation required,Some work needed,Work required throughout
0,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
2,0,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1020,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1021,0,0,0,1,1,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1022,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0


In [17]:
labels

,Completed,Price
0,Not Completed,140000.0
1,Completed,325000.0
2,Completed,500000.0
3,Not Completed,170000.0
4,Completed,410000.0
...,...,...
1019,Not Completed,1200.0
1020,Completed,340000.0
1021,Not Completed,450000.0
1022,Completed,750.0


# Create a class for preparing data

In [ ]:
class PropertyDataset(Dataset):
    def __init__(self, general, room, categorical, labels):
        self.general = general
        self.room = room
        self.categorical = categorical
        self.labels = labels

        self.complete_encoder = LabelEncoder()
        self.complete_encoder.fit(self.labels.Completed)

    def __getitem__(self, item):
